<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/finetuning/embeddings/finetune_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune Embeddings

In this notebook, we show users how to finetune their own embedding models.

We go through three main sections:
1. Preparing the data (our `generate_qa_embedding_pairs` function makes this easy)
2. Finetuning the model (using our `SentenceTransformersFinetuneEngine`)
3. Evaluating the model on a validation knowledge corpus

<b> If you face any errors in running this notebook, you run the code mentioned in the below link in the google colab <b>

https://docs.llamaindex.ai/en/stable/examples/finetuning/embeddings/finetune_embedding/

pip install llama-index-finetuning

## Generate Corpus

First, we create the corpus of text chunks by leveraging LlamaIndex to load some financial PDFs, and parsing/chunking into plain text chunks.

In [1]:
import json

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode

Download Data

download the 'uber_10k' and 'lyft_10k' data files from the below links and move them into the folder that contains this notebook

https://github.com/run-llama/llama_index/blob/9607a05a923ddf07deee86a56d386b42943ce381/docs/docs/examples/data/10k/uber_2021.pdf
https://github.com/run-llama/llama_index/blob/9607a05a923ddf07deee86a56d386b42943ce381/docs/docs/examples/data/10k/lyft_2021.pdf

In [11]:
TRAIN_FILES = ["./lyft_2021.pdf"]
VAL_FILES = ["./uber_2021.pdf"]

TRAIN_CORPUS_FPATH = "./data/train_corpus.json"
VAL_CORPUS_FPATH = "./data/val_corpus.json"

In [12]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

We do a very naive train/val split by having the Lyft corpus as the train dataset, and the Uber corpus as the val dataset.

In [13]:
train_nodes = load_corpus(TRAIN_FILES, verbose=True)
val_nodes = load_corpus(VAL_FILES, verbose=True)

Loading files ['./lyft_2021.pdf']
Loaded 238 docs


Parsing nodes:   0%|          | 0/238 [00:00<?, ?it/s]

Parsed 341 nodes
Loading files ['./uber_2021.pdf']
Loaded 307 docs


Parsing nodes:   0%|          | 0/307 [00:00<?, ?it/s]

Parsed 406 nodes


### Generate synthetic queries

Now, we use an LLM (gpt-3.5-turbo) to generate questions using each text chunk in the corpus as context.

Each pair of (generated question, text chunk used as context) becomes a datapoint in the finetuning dataset (either for training or evaluation).

In [25]:
from llama_index.finetuning import generate_qa_embedding_pairs

PydanticCustomError: Invalid python path: No module named 'pydantic.deprecated.decorator'

In [ ]:
train_dataset = generate_qa_embedding_pairs(train_nodes)
val_dataset = generate_qa_embedding_pairs(val_nodes)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 418/418 [14:54<00:00,  2.14s/it]


In [ ]:
train_dataset.save_json("train_dataset.json")
val_dataset.save_json("val_dataset.json")

In [23]:
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

In [21]:
# [Optional] Load
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

In [22]:
list(train_dataset.queries.values())[1]

'Based on the information provided, how many shares of Class A common stock and Class B common stock did Lyft, Inc. have outstanding on February 22, 2022?'

## Run Embedding Finetuning

In [26]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

PydanticCustomError: Invalid python path: No module named 'pydantic.deprecated.decorator'

In [ ]:
finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id="BAAI/bge-small-en",
    model_output_path="test_model",
    val_dataset=val_dataset,
)

In [ ]:
finetune_engine.finetune()

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/71 [00:00<?, ?it/s]

Iteration:   0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
embed_model = finetune_engine.get_finetuned_model()

## Evaluate Finetuned Model

In this section, we evaluate 3 different embedding models:
1. proprietary OpenAI embedding,
2. open source `BAAI/bge-small-en`, and
3. our finetuned embedding model.

We evaluate the models using **hit rate** metric

We show that finetuning on synthetic (LLM-generated) dataset significantly improve upon an opensource embedding model.

In [27]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

### Define eval function

**Option 1**: We use a simple **hit rate** metric for evaluation:
* for each (query, relevant_doc) pair,
* we retrieve top-k documents with the query,  and
* it's a **hit** if the results contain the relevant_doc.

This approach is very simple and intuitive, and we can apply it to both the proprietary OpenAI embedding as well as our open source and fine-tuned embedding models.

In [ ]:
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(
        nodes, embed_model=embed_model, show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

### Run Evals

#### OpenAI

Note: this might take a few minutes to run since we have to embed the corpus and queries

In [ ]:
embed_model = OpenAIEmbedding(model='text-embedding-3-small')
val_results = evaluate(val_dataset, embed_model)

Generating embeddings:   0%|          | 0/427 [00:00<?, ?it/s]

  0%|          | 0/854 [00:00<?, ?it/s]

In [ ]:
df_opanai = pd.DataFrame(val_results)

In [ ]:
hit_rate = df_opanai["is_hit"].mean()
hit_rate

0.8969555035128806

### BAAI/bge-small-en

In [ ]:
bge = "local:BAAI/bge-small-en"
bge_val_results = evaluate(val_dataset, bge)

Generating embeddings:   0%|          | 0/427 [00:00<?, ?it/s]

  0%|          | 0/854 [00:00<?, ?it/s]

In [ ]:
df_bge = pd.DataFrame(bge_val_results)

In [ ]:
hit_rate_bge = df_bge["is_hit"].mean()
hit_rate_bge

0.8067915690866511

### Finetuned

In [ ]:
finetuned = "local:test_model"
val_results_finetuned = evaluate(val_dataset, finetuned)

Generating embeddings:   0%|          | 0/427 [00:00<?, ?it/s]

  0%|          | 0/854 [00:00<?, ?it/s]

In [ ]:
df_finetuned = pd.DataFrame(val_results_finetuned)

In [ ]:
hit_rate_finetuned = df_finetuned["is_hit"].mean()
hit_rate_finetuned

0.8512880562060889

### Summary of Results

#### Hit rate

In [ ]:
df_ada["model"] = "ada"
df_bge["model"] = "bge"
df_finetuned["model"] = "fine_tuned"

We can see that fine-tuning our small open-source embedding model drastically improve its retrieval quality (even approaching the quality of the proprietary OpenAI embedding)!

In [ ]:
df_all = pd.concat([df_ada, df_bge, df_finetuned])
df_all.groupby("model").mean("is_hit")

,is_hit
model,
ada,0.896956
bge,0.806792
fine_tuned,0.851288


In [ ]:
def build_nodes(filepath):

    reader = SimpleDirectoryReader(input_files=[filepath])
    docs = reader.load_data()
    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=True)

    return nodes

In [ ]:
def build_index(nodes, embed_model):
    index = VectorStoreIndex(
        nodes, embed_model=embed_model, show_progress=True
    )
    return index

In [ ]:
nodes = build_nodes("./data/10k/uber_2021.pdf")

Parsing documents into nodes:   0%|          | 0/307 [00:00<?, ?it/s]

In [ ]:
finetuned_embed_index = build_index(nodes, "local:test_model")
base_embed_index = build_index(nodes, "local:BAAI/bge-small-en")
openai_embed_index = build_index(nodes, ada)

Generating embeddings:   0%|          | 0/427 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/427 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/427 [00:00<?, ?it/s]

In [ ]:
finetuned_embed_qe = finetuned_embed_index.as_query_engine(similarity_top_k=2)
base_embed_qe = base_embed_index.as_query_engine(similarity_top_k=2)
openai_embed_qe = openai_embed_index.as_query_engine(similarity_top_k=2)

In [ ]:
query = "what is the revenue of uber?"
response1 = finetuned_embed_qe.query(query)
response2 = base_embed_qe.query(query)
response3 = openai_embed_qe.query(query)

In [ ]:
print(response1)

In [ ]:
print(response2)

In [ ]:
print(response3)

In [ ]:
for node in response1.source_nodes:
    print("NODE")
    print(node.get_text())
    print("-----")

In [ ]:
for node in response2.source_nodes:
    print("NODE")
    print(node.get_text())
    print("-----")

In [ ]:
for node in response3.source_nodes:
    print("NODE")
    print(node.get_text())
    print("-----")